This file creates a hexagonal grid around the US and assigns both the fire data and weather data to polygons within that grid.
It then outputs a dataframe that has aggregate weather data and fire data for each hex.

In [10]:
#This function gives a hexagonal grid around a certain long, lat based on a specified ring size and resolution

import h3
import geopandas as gpd
import shapely.geometry

def get_hexagon_grid(latitude, longitude, resolution, ring_size):
    """
    Generate a hexagonal grid GeoDataFrame centered around a specified location.

    Parameters:
    - latitude (float): Latitude of the center point.
    - longitude (float): Longitude of the center point.
    - resolution (int): H3 resolution for hexagons.
    - ring_size (int): Number of rings to create around the center hexagon (this will define the radius).

    Returns:
    - hexagon_df (geopandas.GeoDataFrame): GeoDataFrame containing hexagons and their geometries.
    """

    # Get the H3 hexagon for the center point
    center_h3 = h3.latlng_to_cell(latitude, longitude, resolution)
    
    # Generate hexagons within a disk around the center (instead of just rings)
    hexagons = list(h3.grid_disk(center_h3, ring_size))  # All hexagons within the distance
    
    # Create a GeoDataFrame with hexagons and their corresponding geometries
    hexagon_geometries = [shapely.geometry.Polygon(h3.cell_to_boundary(hexagon)) for hexagon in hexagons]
    
    # Create the GeoDataFrame
    hexagon_df = gpd.GeoDataFrame({'Hexagon_ID': hexagons, 'geometry': hexagon_geometries})

    return hexagon_df


In [11]:
#This function creates the hexagonal 

import geopandas as gpd
from shapely.geometry import Point

#This is a sped up version of the previous function
def calculate_hexagon_ids(df, hexagon_df):
    """
    Generates hexagon ids based on the previously calculated hexagonal grid 

    Parameters:
    - df (pandas.DataFrame): The dataframe that we want to add hexagon ids to
    - hexagon_df (geopandas.GeoDataFrame): Dataframe containing h3 hexes

    Returns:
    - df: Dataframe with an additional 'Hexagon_ID' column
    """
    
    # Convert hotel DataFrame to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Latitude'], df['Longitude']),  crs="EPSG:4326")

    if gdf_points.crs != hexagon_df.crs:
       gdf_points = gdf_points.to_crs(hexagon_df.crs)
    # Perform spatial join
    joined = gpd.sjoin(gdf_points, hexagon_df, how='left', predicate='intersects')
    print(joined)

    # Return only the original DataFrame with the new Hexagon_ID column
    df['Hexagon_ID'] = joined['Hexagon_ID'].values

    return df


In [12]:
# Creation of the hexagonal grid 
#The input lat and lng are where the grid is centered- this can be moved to any lat, long - for example, over Califonia (lng = -119.45, lat = 37.17) 
input_lng = -103
input_lat = 44

# Generate H3 hexagons at a specified resolution - a larger resolution means smaller cell size
resolution = 3

# Indicate the number of rings around the central hexagon - larger ring size means more area of the map will be covered
ring_size = 25

# Hexagon grid around a given area
hexagon_df = get_hexagon_grid(input_lat, input_lng, resolution, ring_size)

# Visualize the first rows of the GeoDataFrame
hexagon_df.head()

,Hexagon_ID,geometry
0,832616fffffffff,"POLYGON ((43.448 -102.287, 43.927 -101.769, 44..."
1,83278bfffffffff,"POLYGON ((44.083 -103.536, 44.565 -103.02, 45...."
2,8326a5fffffffff,"POLYGON ((43.037 -103.676, 43.526 -103.166, 44..."
3,832612fffffffff,"POLYGON ((42.396 -102.439, 42.883 -101.928, 43..."
4,832610fffffffff,"POLYGON ((42.797 -101.059, 43.274 -100.541, 43..."


In [13]:
#This code is only used to visualize the hexagonal map - it's not actually needed
import folium
import shapely.geometry

gdf = hexagon_df
m = folium.Map(location=[37.5, -119.5], zoom_start=10)

# Add each hexagon polygon to the map
for _, row in gdf.iterrows():
    # Extract coordinates from the polygon geometry
    polygon = row['geometry']
    
    # Track polygon coordinates of the outside of the polygon
    coordinates = [(lon, lat) for lon, lat in polygon.exterior.coords]
    
    # Add the polygon to the map
    folium.Polygon(
        locations=coordinates,
        color='blue',
        weight=2,
        fill=True,
        fill_opacity=0.4
    ).add_to(m)

# Save the map to an HTML file and show it
m.save("hexagons_map.html")

m

In [14]:
#Importing the fire data
import pandas as pd
import pyarrow.parquet as pq

# Reading in CSV - change path
file_path = 'C:/Users/natha/OneDrive - Iowa State University/Dole, Dhruv S\'s files - DS4010/data/clean/fire_occurrence_point/National_USFS_Fire_Occurrence_Point_(Feature_Layer).parquet'  
fire = pq.read_table(file_path)

fire = fire.to_pandas()

#May have run this if dataframe contains Latitude and Longitude instead of geometry
#fire = fire.rename(columns={'LATDD83': 'Latitude', 'LONGDD83': 'Longitude'})

fire = fire.head(100)

#May not have to run this if dataframe already contains Latitude and Longitude
from shapely.wkb import loads
fire['geometry'] = fire['geometry'].apply(loads)

fire['Latitude'] = fire['geometry'].apply(lambda x: x.y)
fire['Longitude'] = fire['geometry'].apply(lambda x: x.x)

fire

,OBJECTID,FIREOCCURID,CN,FIRENAME,UNIQFIREID,SIZECLASS,TOTALACRES,STATCAUSE,COMMENTS,FIRETYPECATEGORY,PERIMEXISTS,geometry,year,month,day,Latitude,Longitude
0,313655329,None,1526571,Blackies Mesa,2014-NMCAF-000026,A,0.10,Lightning,None,WF,N,POINT (-106.40689999957927 36.288330000104395),2014,07,03,36.288330,-106.406900
1,313655330,None,1523706,3 Tree,2014-NMCAF-000027,B,0.50,Lightning,None,WF,N,POINT (-107.24360000079399 36.97472000088236),2014,06,06,36.974720,-107.243600
2,313655331,None,1525092,Headache,2014-NMCAF-000045,A,0.10,Lightning,None,WF,N,POINT (-107.2860999991065 36.58638999922823),2014,07,28,36.586390,-107.286100
3,313655332,None,1526607,Rito Del Medio,2014-NMCAF-000046,A,0.25,Lightning,None,WF,N,POINT (-106.44469999935627 36.5438900001169),2014,08,19,36.543890,-106.444700
4,313655333,None,1525366,La Cueva,2014-NMCAF-000047,A,0.10,Debris/Open Burning,"Moved to PLSS Location, Still not on Forest",WF,N,POINT (-106.05935680783841 36.281557238716026),2014,08,26,36.281557,-106.059357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,313655424,None,202518,Unknown,1989-NMCAF-000067,D,109.20,Lightning,POLYGON > 10 AC,WF,Y,POINT (-106.24219055979677 36.5800213565829),1989,10,21,36.580021,-106.242191
96,313655425,B72DC3F1-3AA4-4E05-A734-C486673A38FD,6908997,Lobo,2018-NMCAF-000596,B,7.00,Lightning,None,WF,N,POINT (-105.58528769751223 36.59667260049631),2018,07,13,36.596673,-105.585288
97,313655426,B6A20DD3-BF51-4339-9983-BDE9D6F17CEE,6893324,Windmill,2018-NMCAF-000615,A,0.10,Lightning,None,WF,N,POINT (-107.25028809013843 36.85917253814278),2018,07,17,36.859173,-107.250288
98,313655427,F3D55C05-AE58-4A6D-8CFA-5557CCFF7256,6908697,Barela Springs,2018-NMCAF-000649,A,0.25,Lightning,None,WF,N,POINT (-106.0766767116782 36.73806148263754),2018,07,23,36.738061,-106.076677


In [15]:
#Importing hexagonal grid to a geodataaframe
hexagon_df = gpd.GeoDataFrame(
    hexagon_df,
    geometry='geometry',
    crs="EPSG:4326"  
)

In [16]:
#Calculating a hexagonal id for each fire
fire = calculate_hexagon_ids(fire, hexagon_df)
fire.head()

     OBJECTID                           FIREOCCURID       CN        FIRENAME  \
0   313655329                                  None  1526571   Blackies Mesa   
1   313655330                                  None  1523706          3 Tree   
2   313655331                                  None  1525092        Headache   
3   313655332                                  None  1526607  Rito Del Medio   
4   313655333                                  None  1525366        La Cueva   
..        ...                                   ...      ...             ...   
95  313655424                                  None   202518         Unknown   
96  313655425  B72DC3F1-3AA4-4E05-A734-C486673A38FD  6908997            Lobo   
97  313655426  B6A20DD3-BF51-4339-9983-BDE9D6F17CEE  6893324        Windmill   
98  313655427  F3D55C05-AE58-4A6D-8CFA-5557CCFF7256  6908697  Barela Springs   
99  313655428  5D7591D6-5155-4C04-A451-5C68E2AECF21  6890369      Jarita Rim   

           UNIQFIREID SIZECLASS  TOTALA

,OBJECTID,FIREOCCURID,CN,FIRENAME,UNIQFIREID,SIZECLASS,TOTALACRES,STATCAUSE,COMMENTS,FIRETYPECATEGORY,PERIMEXISTS,geometry,year,month,day,Latitude,Longitude,Hexagon_ID
0,313655329,None,1526571,Blackies Mesa,2014-NMCAF-000026,A,0.10,Lightning,None,WF,N,POINT (-106.40689999957927 36.288330000104395),2014,07,03,36.288330,-106.406900,8348dbfffffffff
1,313655330,None,1523706,3 Tree,2014-NMCAF-000027,B,0.50,Lightning,None,WF,N,POINT (-107.24360000079399 36.97472000088236),2014,06,06,36.974720,-107.243600,832699fffffffff
2,313655331,None,1525092,Headache,2014-NMCAF-000045,A,0.10,Lightning,None,WF,N,POINT (-107.2860999991065 36.58638999922823),2014,07,28,36.586390,-107.286100,8348d9fffffffff
3,313655332,None,1526607,Rito Del Medio,2014-NMCAF-000046,A,0.25,Lightning,None,WF,N,POINT (-106.44469999935627 36.5438900001169),2014,08,19,36.543890,-106.444700,8348dbfffffffff
4,313655333,None,1525366,La Cueva,2014-NMCAF-000047,A,0.10,Debris/Open Burning,"Moved to PLSS Location, Still not on Forest",WF,N,POINT (-106.05935680783841 36.281557238716026),2014,08,26,36.281557,-106.059357,8348dbfffffffff


In [17]:
#Importing the weather dataframe
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import pyarrow.compute as pc

#Change path
file_path = 'C:/Users/natha/OneDrive - Iowa State University/Dole, Dhruv S\'s files - DS4010/data/clean/ghcnd/daily/2020.parquet'  
weather = pq.read_table(file_path)

weather = weather.to_pandas()

#Converting to a geopandas dataframe
weather['geometry'] = weather['geometry'].apply(loads)

weather['Latitude'] = weather['geometry'].apply(lambda x: x.y)
weather['Longitude'] = weather['geometry'].apply(lambda x: x.x)

In [18]:
#Calculating hexagon ids for the weather data
temp = calculate_hexagon_ids(weather, hexagon_df)
temp.head()

          station_id  year  month  day   prcp  snow  snwd   tmax   tmin  awnd  \
0        CA001011500  2020      1    1    0.0   0.0   0.0  130.0   65.0   NaN   
1        CA001011500  2020      1    2  322.0   0.0   0.0   65.0   20.0   NaN   
2        CA001011500  2020      1    3  121.0   0.0   0.0  105.0   10.0   NaN   
3        CA001011500  2020      1    4  146.0   0.0   0.0   70.0   30.0   NaN   
4        CA001011500  2020      1    5   63.0   0.0   0.0   75.0   35.0   NaN   
...              ...   ...    ...  ...    ...   ...   ...    ...    ...   ...   
8647344  USW00096409  2020     12   27    0.0   NaN   NaN -116.0 -213.0   NaN   
8647345  USW00096409  2020     12   28    0.0   NaN   NaN  -89.0 -166.0   NaN   
8647346  USW00096409  2020     12   29    0.0   NaN   NaN  -95.0 -189.0   NaN   
8647347  USW00096409  2020     12   30    0.0   NaN   NaN -159.0 -248.0   NaN   
8647348  USW00096409  2020     12   31    0.0   NaN   NaN -189.0 -257.0   NaN   

         elevation state   

,station_id,year,month,day,prcp,snow,snwd,tmax,tmin,awnd,elevation,state,geometry,Latitude,Longitude,Hexagon_ID
0,CA001011500,2020,1,1,0.0,0.0,0.0,130.0,65.0,NaN,75.0,BC,POINT (-123.75 48.9333),48.9333,-123.75,8328dcfffffffff
1,CA001011500,2020,1,2,322.0,0.0,0.0,65.0,20.0,NaN,75.0,BC,POINT (-123.75 48.9333),48.9333,-123.75,8328dcfffffffff
2,CA001011500,2020,1,3,121.0,0.0,0.0,105.0,10.0,NaN,75.0,BC,POINT (-123.75 48.9333),48.9333,-123.75,8328dcfffffffff
3,CA001011500,2020,1,4,146.0,0.0,0.0,70.0,30.0,NaN,75.0,BC,POINT (-123.75 48.9333),48.9333,-123.75,8328dcfffffffff
4,CA001011500,2020,1,5,63.0,0.0,0.0,75.0,35.0,NaN,75.0,BC,POINT (-123.75 48.9333),48.9333,-123.75,8328dcfffffffff
